# Linking Market IDs to FactSet Entity IDs

The FactSet Supply Chain Relationships DataFeed provides data for FactSet company identifiers, factset_entity_id.  This notebook will cover how to link a market identifier for a security to the required FactSet ID within Supply Chain Relationships. 

## Notice
This example workbook provides queries in SQL formatted for MSSQL and sample python code. It may be necessary to adjust the query language to suit a different SQL DBMS. Sample queries and code should be viewed as possible examples of data calculations and are subject to change. The queries and code provided should be used as a guide to understand how the underlying data items can be used but are not guaranteed to represent the same methodology as the FactSet Workstation or other industry calculations. Queries cannot be guaranteed to be written for performance and/or efficiency.

## Import Packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyodbc
import os


import sys
sys.path.append('../')
import utils.loadsql as loadsql
sys.path.append('/')

#Define Directory of SQL Queries
sql_path = './../SQL Queries/'

## Create DSN Connection
Enter a DSN associated with the MSSQL database containing FactSet Standard DataFeeds.

In [2]:
dsn = 'SDF'
cxn = pyodbc.connect('DSN={dsn_name}'.format(dsn_name = dsn))

## Linking a Market ID to FactSet Symbology

### Key Symbology Tables

For a full overview of Symbology Tables and linking from various types of market identifiers, please check out the **Symbology 1.2 Linking Market ID to fsym_id** notebook found in the **FactSet/FactSet/Symbology V1/Jupyter Notebooks** directory. The examples here utilize current tickers from the SYM_TICKER_REGION table, please see the Symbology notebooks for a discussion of historical Ticker Symbols.  

For this example, we show how to navigate from a Ticker to the FactSet company identifier, factset_entity_id. 

Key table used:
* **SYM_TICKER_REGION**: Maps Ticker-Region to corresponding Regional fsym_id (-R).

In order to link FactSet security Ids and FactSet company Ids we will use:

* **ENT_SCR_SEC_ENTITY_HIST**: This table maps fsym_ids to factset_entity_ids for the Supply Chain Relationships universe. The mappings are provided at the Security fsym_id (-S). Alongside each symbol pair, a start and end date is provided denoting the window where FactSet considers the association valid. If the relationship is current, an end date of Null will be provided.

#### SQL Example

The following query retrieves the FactSet company identifier, factset_entity_id, based on the specified ticker.  For this example, we will use S-US.

The flow is as follows:

* **SYM_TICKER_REGION**: This table is used to translate a regional ticker to the Regional FactSet Permanent ID (fsym_id).

* **SYM_COVERAGE**: This table is a master symbology table providing metadata around the given fsym_id, including linking Regional fsym_ids to Security fsym_ids.

* **ENT_SCR_SEC_ENTITY_HIST**: This table is used to translate the Security fsym_id to the factset_entity_id, providing the start and end date when this security to entity relationship was valid. 

**Ticker**  &rarr; **fsym_id** &rarr; **fsym_security_id** &rarr; **factset_entity_id** 

**S-US** &rarr; **XKDSRD-R** &rarr; **BB0C2F-S** &rarr; **0D7N1X-E**

In [3]:
ticker = 'S-US'

#loadsql is a cover function used to retrieve SQL queries from a directory
q = loadsql.get_sql_q(os.path.join(sql_path,'1.2.1 Supply Chain - Linking a Market ID to a FactSet ID.sql')
                      ,show=0,connection=dsn).format(sym=ticker)

print(q)
tick_ent = pd.read_sql(q,cxn)
tick_ent

DECLARE @id NVARCHAR(50)= 'S-US';
SELECT tr.ticker_region, 
       cov.fsym_id, 
       cov.proper_name, 
       e.entity_proper_name, 
       se.fsym_id, 
       se.start_date, 
       se.end_date, 
       se.factset_entity_id
FROM   sym_v1.sym_ticker_region AS tr
JOIN sym_v1.sym_coverage AS cov
  ON tr.fsym_id = cov.fsym_id
JOIN ent_v1.ent_scr_sec_entity_hist AS se
  ON se.fsym_id = cov.fsym_security_id
JOIN sym_v1.sym_entity AS e
  ON e.factset_entity_id = se.factset_entity_id
WHERE  tr.ticker_region = @id;


,ticker_region,fsym_id,proper_name,entity_proper_name,fsym_id,start_date,end_date,factset_entity_id
0,S-US,XKDSRD-R,Sprint Corp.,Sprint Corp.,BB0C2F-S,2013-07-08,None,0D7N1X-E


In this example, the entity ID for Sprint Corp. is returned. The ENT_SCR_SEC_ENTITY_HIST table unlocks the ability to track the entities linked to a given fsym_id through time. If the entity ID tied to a given fsym_id changed over time, multiple entity IDs will be returned unless the start_date and end_dates are specified. 

## Next Steps

* **Please check out these notebooks for additional information on the FactSet Supply Chain Relationships DataFeed:** 
    * 1.1 Introduction to FactSet Supply Chain Relationships
    * 1.3 Mapping Customers and Suppliers to Tradable Securities


* **Documentation for the feeds discussed above can be found within the** [Open:FactSet Marketplace](https://open.factset.com/en-us)
    * [FactSet Supply Chain Relationships](https://open.factset.com/products/factset-supply-chain-relationships/en-us) 
 
 
* **Please direct any training requests or questions to your CTS Implementation Specialist.**